<a href="https://colab.research.google.com/github/Benchamin242/Scraping/blob/main/MatchScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


In [3]:
data = requests.get(standings_url)


In [4]:
from bs4 import BeautifulSoup


In [5]:
soup = BeautifulSoup(data.text)


In [6]:
standings_table = soup.select('table.stats_table')[0]


In [7]:
links = standings_table.find_all('a') #find the link to the squads


In [8]:
links = [l.get("href") for l in links]

In [9]:
links = [l for l in links if '/squads/' in l] #filter to just find the squad linnk only

In [10]:
team_urls = [f"https://fbref.com{l}" for l in links] #provide absolute path so we can download data

In [11]:
data = requests.get(team_urls[0])

In [12]:
import pandas as pd

In [14]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

<ipython-input-14-c28466f6a61e>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0] #parse html so we can use it later


In [15]:
soup = BeautifulSoup(data.text)

In [17]:
links = soup.find_all('a')


In [18]:
links = [l.get("href") for l in links]


In [20]:
links = [l for l in links if l and 'all_comps/shooting/' in l] #filters it to only find links to shooting

In [21]:
data = requests.get(f"https://fbref.com{links[0]}") #downloads our data

In [22]:
shooting = pd.read_html(data.text, match="Shooting")[0]

<ipython-input-22-8da1f2d296b3>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [23]:
shooting.head()

For Liverpool                                                              \
           Date   Time            Comp         Round  Day Venue Result   GF   
0    2024-08-17  12:30  Premier League   Matchweek 1  Sat  Away      W  2.0   
1    2024-08-25  16:30  Premier League   Matchweek 2  Sun  Home      W  2.0   
2    2024-09-01  16:00  Premier League   Matchweek 3  Sun  Away      W  3.0   
3    2024-09-14  15:00  Premier League   Matchweek 4  Sat  Home      L  0.0   
4    2024-09-17  21:00    Champions Lg  League phase  Tue  Away      W  3.0   

                         ... Standard             Expected                    \
    GA         Opponent  ...     Dist FK PK PKatt       xG npxG npxG/Sh G-xG   
0  0.0     Ipswich Town  ...     14.8  0  0     0      2.6  2.6    0.15 -0.6   
1  0.0        Brentford  ...     13.6  1  0     0      2.5  2.5    0.14 -0.5   
2  0.0   Manchester Utd  ...     13.4  0  0     0      1.8  1.8    0.16  1.2   
3  1.0  Nott'ham Forest  ...     14.9  0  0     0      0.9  0.9    0.07 -0.9   
4  1.0         it Milan  ...     15.7  1  0     0      3.1  3.1    0.14 -0.1   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -0.6        Match Report  
1    -0.5        Match Report  
2     1.2        Match Report  
3    -0.9        Match Report  
4    -0.1        Match Report  

[5 rows x 26 columns]

In [24]:
shooting.columns = shooting.columns.droplevel() #drops unnecessary level

In [25]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date") #merges shooting data and match data

In [26]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,4-2-3-1,Tim Robinson,Match Report,NaN,18,5,14.8,0,0,0
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,4-4-2,Stuart Attwell,Match Report,NaN,19,8,13.6,1,0,0
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,3,13.4,0,0,0
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,4-2-3-1,Michael Oliver,Match Report,NaN,14,5,14.9,0,0,0
4,2024-09-17,21:00,Champions Lg,League phase,Tue,Away,W,3.0,1.0,it Milan,...,4-2-3-1,Espen Eskås,Match Report,NaN,23,11,15.7,1,0,0


In [27]:
years = list(range(2024, 2022, -1)) #allows scraping of last 2 years

In [28]:
all_matches = [] #will contain several datframes of matchlogs for 1 team

In [29]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")